<a href="https://colab.research.google.com/github/lstarke/Health-Insurance-Cost-Prediction/blob/main/LeandroStarke_Health_Insurance_Cost_Prediction_(AutoML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Business Understanding

## Sobre o problema - Health Insurance Cost

<center><img src="https://drive.google.com/uc?export=view&id=1vwwSzYFJxMxSEtPktnq96J6y3H1hAmX0" width="80%"></p></center>

Neste projeto será utilizado um dataset disponivel no Kaggle chamado [Health Insurance Cost Prediction](https://www.kaggle.com/datasets/annetxu/health-insurance-cost-prediction). Nele temos informações sobre pacientes de um plano de saúde e seus respectivos custos médicos. Ele inclui 1338 entradas com 7 variáveis:

* `age`: idade do paciente
* `sex`: gênero do paciente (masculino ou feminino)
* `bmi`: Índice de Massa Corporal (IMC) do paciente
* `children`: número de filhos dependentes do paciente
* `smoker`: informação se o paciente é fumante ou não
* `region`: região dos Estados Unidos em que o paciente reside (nordeste, sudeste, sudoeste ou noroeste)
* `charges`: custo médico individual do paciente durante um ano

O objetivo do conjunto de dados é prever o custo médico de um paciente com base em suas características pessoais, um problema de regressão, onde a variável alvo é `charges`.

Esse conjunto de dados é interessante porque pode ajudar a compreender como as características pessoais de um paciente podem influenciar nos custos médicos. Além disso, ele pode ser utilizado para criar modelos de precificação de seguros de saúde ou para identificar pacientes com alto risco de custos médicos.

## Sobre Auto Machine Learning (AutoML)

<center><img src="https://drive.google.com/uc?export=view&id=1vB46w2YoEmLKSpxqx4aRh2wbZOM_TTk7" width="80%"></p></center>

Uma ferramenta útil para cientistas de dados e desenvolvedores que desejam criar rapidamente modelos de aprendizado de máquina e acelerar o processo de experimentação de modelos.

PyCaret é uma biblioteca de auto machine learning (AutoML) de código aberto que visa simplificar o processo de criação de modelos de aprendizado de máquina. Ela fornece uma interface de usuário intuitiva e fácil de usar para realizar tarefas comuns de ciência de dados, como pré-processamento de dados, seleção de recursos, validação cruzada e ajuste de hiperparâmetros.

Com o PyCaret, é possível construir rapidamente vários modelos de aprendizado de máquina, desde modelos de regressão e classificação até modelos de clustering e detecção de anomalias. Além disso, a biblioteca inclui recursos como a criação de fluxos de trabalho de ponta a ponta, seleção automática de recursos e explicações de modelo.

# Instalação de pacotes

In [1]:
# descomentar linhas abaixo quando rodar a primeira vez
# !pip install matplotlib==3.6.3
# !pip install -U pandas-profiling
# !pip install pycaret -q

# Importação de pacotes

In [ ]:
# importar os pacotes necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pandas_profiling import ProfileReport
from sklearn.preprocessing import KBinsDiscretizer
from pycaret.utils import enable_colab 


# habilitando o pycaret no colab
enable_colab()

# Data Understanding

In [ ]:
# importar os dados
DATA_PATH = "/content/insurance.csv"
df = pd.read_csv(DATA_PATH)

# ver as 5 primeiras entradas
df.head()

In [ ]:
# inspecionando quantidade de linhas e colunas do DataFrame
df.shape

In [ ]:
profile = ProfileReport(df, title="Insurance EDA")
profile

## Conclusões Pandas Profiling



* Não há dados falstantes no DataFrame;
* Idade mínima e máxima são 18 e 64, respectivamente;
* Há uma maior concentração de pessoas com idade abaixo de 20 anos;
* A porporção de sexo masculino e feminino varia somente 1%;
* A distribuição da variável `bmi` (IMC) é normal;
* A distribuição da variável `children` é assimétrica positiva;
* Aproximadamente 43% das pessoas não tem filhos;
* 20% das pessoas são fumantes;
* A distribuição da variável `charges` é assimétrica positiva;






# Data Preparation

## KBinsDiscretizer

KBinsDiscretizer é um transformador de pré-processamento de dados da biblioteca scikit-learn (sklearn.preprocessing) que divide determinada feature em intervalos (bins) discretos e categóricos com base em um número especificado de bins. 

Isso pode ser útil em algumas situações em que você deseja converter uma característica contínua em uma característica categórica. Por exemplo, se você estiver trabalhando com dados de idade, pode ser interessante discretizar as idades em faixas etárias (por exemplo, 0-10, 10-20, 20-30, etc.) em vez de manter a idade como uma característica contínua.

O KBinsDiscretizer usa a estratégia de "uniform" ou "quantile" para definir os limites dos bins. Quando a estratégia é "uniform", todos os bins têm a mesma largura. Quando a estratégia é "quantile", cada bin contém aproximadamente o mesmo número de amostras.

Definimos o parâmetro encode como ordinal, assim, cada bin é representado por um único número inteiro, que indica a ordem do bin. A primeira ordem é 0, a segunda ordem é 1, e assim por diante. Cada característica original é substituída por um vetor de números inteiros correspondente aos bins discretizados. Essa opção é útil quando existe uma relação ordinal entre os bins.

In [ ]:
# definindo o número de intervalos
n_bins = 3

# criando o objeto KBinsDiscretizer
discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile')

# atribuindo uma nova coluna ao DataFrame contendo o identificador do grupo ao qual a pessoa pertence
df = df.assign(age_discretized = discretizer.fit_transform(df.age.values.reshape(-1, 1)))
df.head()

Abaixo podemos ver melhor o que o KBinsDiscretizer faz.

1. Grupo 1: Somente pessoas com idade entre 18 e 30 anos;
2. Grupo 2: Somente pessoas com idade entre 31 e 46 anos;
3. Grupo 3: Somente pessoas com idade entre 47 e 64 anos;



In [ ]:
# verificando valores unicos na coluna age_discretized
bins = df.age_discretized.unique()

# laço sobre a lista de bins e imprimindo os valores mínmos e máximos encontrados dentro de cada bin
for bin in bins:
  print(f"bin = {int(bin)}: {df[df.age_discretized == bin].age.min()} - {df[df.age_discretized == bin].age.max()}")

In [ ]:
# instanciando os objetos fig e ax
fig, ax = plt.subplots(figsize=(10, 6))

# criando um gráfico de barras com a contagem de pessoas em cada intervalo de age_discretized
df.age_discretized.value_counts().plot(kind="bar", ax=ax)

# colocando a quantidade em cada intervalo
for c in ax.containers:
  ax.bar_label(c, label_type="center")
  
# ajusta o layout do gráfico para garantir que todos os elementos estejam visíveis
plt.tight_layout()

Temos uma distrição muito parecida entre os grupos de ```age_discretized```.

In [ ]:
# separando os dados em teste e treino
test = df.sample(frac=0.10)
train = df.drop(test.index)

test.reset_index(inplace=True, drop=True)
train.reset_index(inplace=True, drop=True)

# Modeling

# Evaluation

# Conclusões

# Referências